In [1]:
import os

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torch.nn.init as weight_init

from sklearn.model_selection import train_test_split

In [15]:
MODEL_DIR = "Model"
MODEL_VER = "1"
MODEL_FILE = "model.pt"
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_VER)

OUTPUT_PATH = "Output"
OUTPUT_FILE = f"output{MODEL_VER}.csv"
OUTPUT_R_FILE = f"output_rating{MODEL_VER}.csv"

OUTPUT_FILE_RATED = f"output_rated{MODEL_VER}.csv"
OUTPUT_R_FILE_RATED = f"output_rating_rated{MODEL_VER}.csv"

if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

# Model

In [3]:
lr = 0.01
dropout = 0.1
epochs = 200
activation = "sigmoid"

#### Reference: https://medium.com/@haoyunlai/pytorch-implementation-of-autoencoder-based-recommender-system-9aff6c3d1b02

In [4]:
class Encoder(nn.Module):
    """
    Class for Symmetrical AutoEncoder Network
    :param L: List of int, contains sizes of encoding layers and starts with feature size
    For example: [500, 20, 10] will result in:
      - encoder 2 layers: 500x20 and 20x10. Representation layer (z) will be 10
      - decoder 2 layers: 10x20 and 20x500, output size is 500, reconstructed.
    :param activation_fn: (default 'sigmoid') Type of activation function
    :param drop_prob: (default: 0.0) Dropout probability
    """
    def __init__(self, L, activation_fn='sigmoid', drop_prob=0.0):
        super(Encoder, self).__init__()
        layers = self.create_nn_structure(L)
        self.num_layers = len(L)
        # create controller for activation function
        self.activation_fn_nm = activation_fn
        # create dropout module
        self._drop_prob = drop_prob
        if drop_prob > 0.0:
            self.dropout = nn.Dropout(drop_prob)
        #initialize with empty list to store layers
        self.linears = nn.ModuleList([])
        self.linears.extend([nn.Linear(i[0], i[1]) for i in layers])
        
    def get_activation_fn(self):
        # user selected activation function at layers except for last layer
        if self.activation_fn_nm == 'relu':
            return nn.ReLU()
        elif self.activation_fn_nm == 'lrelu':
            return nn.LeakyReLU()
        elif self.activation_fn_nm == 'sigmoid':
            return nn.Sigmoid()
        else:
            raise ValueError('Activation function type not defined')
    
    def forward(self, x):
        for i,layer in enumerate(self.linears):
            if i <= self.num_layers-1:
                # create instance of activation function
                act_fn = self.get_activation_fn()
                # pass in the input
                x = act_fn(self.linears[i](x))
                if self._drop_prob > 0.0 and i <= int(self.num_layers/2): 
                    # apply dropout only on encode layer by control of i
                    x = self.dropout(x)
        # No activation on the last decoding layer
        x = self.linears[-1](x)
        return x

    def create_nn_structure(self, L):
        max_ind = len(L)-1
        layers = []
        for i,v in enumerate(L):
            if i < max_ind:
                #still have i+1 available, create layer tuple
                layer = [v,L[i+1]]
                layers.append(layer)
        #then inverse the layers for decoder size
        encoder_layers = layers[:]
        for l in encoder_layers[::-1]:
            decoder_layer = l[::-1]
            layers.append(decoder_layer)
        return layers

### Data Preprocessing methods

In [5]:
def create_index_mapping(L):
    '''
    return reindexed dict on user and items
    encoded indices starts from 1
    input: 
    * L: list of str
    outputs:
    * ind_2_item,item_2_ind: tuple of dictionary
    '''
    L = set(L)
    ind_2_item = {}
    
    for i,v in enumerate(L):
        #index start from 1
        ind_2_item[i+1] = v
        
    #invert the map
    item_2_ind = {v: k for k, v in ind_2_item.items()}
    return ind_2_item,item_2_ind

def reindexer(ratings_df,user_col,item_col,rating_col):
    '''
    inputs:
    * ratings_df: pandas df containing ratings/affinity for user-item pairs
    * user_col: actual col name for users
    * item_col: actual col name for items
    * rating_col: actual col name for ratings
    output:
    * ratings_df: reindexed user and item column, pandas df
    '''
    users_list = ratings_df[user_col].tolist()
    item_list = ratings_df[item_col].tolist()
    
    ind_2_user,user_2_ind = create_index_mapping(users_list)
    ind_2_item,item_2_ind = create_index_mapping(item_list)
    
    #rename ratings df columns
    ratings_df = ratings_df.rename(columns={user_col:'user_col',
                                            item_col:'item_col',
                                            rating_col:'rating_col'})
    
    #encode df using the 2 mappings
    ratings_df['encoded_users'] = ratings_df['user_col'].apply(lambda x:user_2_ind[x])
    ratings_df['encoded_items'] = ratings_df['item_col'].apply(lambda x:item_2_ind[x])
    
    return ratings_df[['encoded_users','encoded_items','rating_col']], ind_2_user,user_2_ind, ind_2_item,item_2_ind

def reindex_movies(movies, item_2_ind):
    movies['movieid'] = movies['movieid'].apply(lambda x:item_2_ind[x] if x in item_2_ind else -1)
    id_2_movie = pd.Series(movies.movie.values,index = movies.movieid).to_dict()

    return id_2_movie

def convert(data):
    new_data = []
    for id_users in range(1,nb_users+1):
        id_items = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_items-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

# Data loading and preprocessing

In [6]:
ratings = pd.read_csv('MovieLens1M/ratings.dat', delimiter = '::',header=None, engine='python')
movies = pd.read_csv('MovieLens1M/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

ratings.columns = ['userid','movieid','rating','timestamp']
ratings_reindex, ind_2_user,user_2_ind , ind_2_item,item_2_ind = reindexer(ratings,'userid','movieid','rating')

movies.columns = ['movieid', 'movie', 'genre']

id_2_movie = reindex_movies(movies, item_2_ind)



train, test = train_test_split(ratings_reindex,
                               stratify=ratings_reindex['encoded_users'],
                               test_size=0.1,
                               random_state=42)

training_set = np.array(train, dtype='int')
test_set = np.array(test, dtype='int')

nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
  
training_set = convert(training_set)
test_set = convert(test_set)

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

print(f"Number of users: {nb_users}, Number of Movies: {nb_movies}")
print(f"Training set size: {training_set.shape}, Validation set size: {test_set.shape}")

Number of users: 6040, Number of Movies: 3706
Training set size: torch.Size([6040, 3706]), Validation set size: torch.Size([6040, 3706])


# Training and Validation

In [7]:
autoencoder_network = Encoder([nb_movies, 20,10], activation, dropout)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(autoencoder_network.parameters(), lr = lr, weight_decay = 0.5)

validationLoss = float("inf")
minEpoch = epochs
nb_epoch = epochs


for epoch in range(1, nb_epoch + 1):
    autoencoder_network.train()
    train_loss = 0
    train_s = 0.
    # s is the number of users who rated at least 1 movies
    for id_user in range(nb_users):
        input_ids = Variable(training_set[id_user]).unsqueeze(0)
        target = Variable(training_set[id_user]).unsqueeze(0)
        
        if torch.sum(target.data > 0) > 0:
            output = autoencoder_network(input_ids)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) #making this anyway not equal to 0, as this will be a denominator
            #mean_corrector is the avg of the error, only considering the movies having ratings (non-zero ratings) for computing mean of error
            loss.backward() # decide the direction the increment of weights
            #this call will just computing all the gradients required
            train_loss += np.sqrt(loss.data*mean_corrector)
            train_s += 1.
            optimizer.step() # decide the amount to update the weights
    
    # Validation
    autoencoder_network.eval()
    test_loss = 0
    test_s = 0.

    res = []
    targets = []

    # averaged difference between real rating and predicted rating
    for id_user in range(nb_users):
        input_ids = Variable(training_set[id_user]).unsqueeze(0) # should keep the training set
        target = Variable(test_set[id_user]).unsqueeze(0) # to predict the other movies user not seen yet

        if torch.sum(target.data > 0) > 0:
            # make predictions
            with torch.no_grad():
                output = autoencoder_network(input_ids)
                targets.append(target.detach().numpy())
                target.require_grad = False
                output[target == 0] = 0 # dont want to measure the loss on the movies didnt get the actual rating from user 
                # force to 0 and difference / loss will be 0 for those entries
                loss = criterion(output, target)

                mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10) 
                # only consider the movies that are rated in the test set, to be included in the loss
                test_loss += np.sqrt(loss.data*mean_corrector)
                test_s += 1.
    print(f"epoch: {epoch} train loss: {train_loss/train_s}|| test loss: {test_loss/test_s}")
    
    # Saving Model
    if validationLoss > (test_loss/test_s):
        validationLoss = (test_loss/test_s)
        minEpoch = epoch
        torch.save(autoencoder_network.state_dict(), os.path.join(MODEL_PATH,MODEL_FILE))

epoch: 1 train loss: 1.2799078226089478|| test loss: 0.9894769191741943
epoch: 2 train loss: 0.9995525479316711|| test loss: 0.9657177329063416
epoch: 3 train loss: 0.9856230020523071|| test loss: 0.9603356122970581
epoch: 4 train loss: 0.980739414691925|| test loss: 0.9572633504867554
epoch: 5 train loss: 0.9787437319755554|| test loss: 0.9562706351280212
epoch: 6 train loss: 0.9775118231773376|| test loss: 0.9559714198112488
epoch: 7 train loss: 0.9765394330024719|| test loss: 0.9549124836921692
epoch: 8 train loss: 0.9761330485343933|| test loss: 0.9542557001113892
epoch: 9 train loss: 0.9758598804473877|| test loss: 0.9538522362709045
epoch: 10 train loss: 0.9756033420562744|| test loss: 0.9546443223953247
epoch: 11 train loss: 0.9753249287605286|| test loss: 0.9538071155548096
epoch: 12 train loss: 0.9750252962112427|| test loss: 0.9545860290527344
epoch: 13 train loss: 0.9748006463050842|| test loss: 0.953475832939148
epoch: 14 train loss: 0.9747345447540283|| test loss: 0.953836

epoch: 114 train loss: 0.9349737763404846|| test loss: 0.9103661775588989
epoch: 115 train loss: 0.9345659613609314|| test loss: 0.9082899689674377
epoch: 116 train loss: 0.9325552582740784|| test loss: 0.9089234471321106
epoch: 117 train loss: 0.9316856265068054|| test loss: 0.9087470173835754
epoch: 118 train loss: 0.9313837885856628|| test loss: 0.9080294966697693
epoch: 119 train loss: 0.9327404499053955|| test loss: 0.9090966582298279
epoch: 120 train loss: 0.9309738278388977|| test loss: 0.9082213044166565
epoch: 121 train loss: 0.9318990111351013|| test loss: 0.9067721366882324
epoch: 122 train loss: 0.9306854605674744|| test loss: 0.9064527750015259
epoch: 123 train loss: 0.9300559163093567|| test loss: 0.9072864651679993
epoch: 124 train loss: 0.9307157397270203|| test loss: 0.908343493938446
epoch: 125 train loss: 0.9304563999176025|| test loss: 0.9092302918434143
epoch: 126 train loss: 0.9312623739242554|| test loss: 0.9093031883239746
epoch: 127 train loss: 0.93001830577850

In [8]:
print(f"RMSE obtained: {validationLoss.data} after {minEpoch} epochs")

RMSE obtained: 0.8904616236686707 after 200 epochs


# Making top k recommendation

In [9]:
# Loading model
autoencoder_network = Encoder([nb_movies, 20,10], activation, dropout)
autoencoder_network.load_state_dict(torch.load(os.path.join(MODEL_PATH,MODEL_FILE)))
autoencoder_network.eval()

evidence = np.array(ratings_reindex, dtype='int')
evidence = convert(evidence)
evidence = torch.FloatTensor(evidence)

In [10]:
def make_top_k_recommendations(encoder,evidence,k,filter_seen=True):
    '''
    :param encoder: autoencoder instance
    :param evidence: full set of seen ratings from all users
    :param k: top k items (by output score)
    :param filter_seen: (default True) filter controller to remove seen items from top k list
    '''     
    res = []
    rated = []
    nb_users = evidence.shape[0]
    # to find top scored items for each user
    for id_user in range(nb_users):
        encoder_input = Variable(evidence[id_user]).unsqueeze(0) # should keep the training set 
        encoder_output = encoder(encoder_input)
        
        target = Variable(evidence[id_user]).unsqueeze(0) # mask to find items not seen yet
        if filter_seen:
            output_rated = encoder_output.clone()
            output_rated[target == 0] = 0
            rated.append(output_rated.detach().numpy())
            encoder_output[target != 0] = 0 # force seen items scores to 0, will never get recommended
            
        res.append(encoder_output.detach().numpy())
        
    res = [a[0] for a in res]
    rated = [a[0] for a in rated]
#     print(rated)
    final_itemsets = []
    rated_itemsets = []
    
    for each, each_r in zip(res,rated):
        full_ratings_predicted = list(each)
        full_ratings_predicted_r = list(each_r)
        full_ratings_indexed = list(enumerate(full_ratings_predicted))
        full_ratings_indexed_r = list(enumerate(full_ratings_predicted_r))
        final_itemsets.append(sorted(full_ratings_indexed,key=lambda x:x[1],reverse =True)[:k])
        rated_itemsets.append(sorted(full_ratings_indexed_r,key=lambda x:x[1],reverse =True)[:k])
        
    return final_itemsets, rated_itemsets



In [11]:
recommendations, recommendations_r = make_top_k_recommendations(autoencoder_network, evidence, 10)

In [12]:
recommendation_movies = [[r[0] for r in recommendation] for recommendation in recommendations]
movies_ratings = [[r[1] for r in recommendation] for recommendation in recommendations]

recommendation_movies_r = [[r[0] for r in recommendation] for recommendation in recommendations_r]
movies_ratings_r = [[r[1] for r in recommendation] for recommendation in recommendations_r]

In [13]:
user_recommendation = pd.DataFrame(recommendation_movies)
user_recommendation = user_recommendation.applymap(lambda x:id_2_movie[x+1])

recommendation_ratings = pd.DataFrame(movies_ratings)

user_recommendation_r = pd.DataFrame(recommendation_movies_r)
user_recommendation_r = user_recommendation_r.applymap(lambda x:id_2_movie[x+1])

recommendation_ratings_r = pd.DataFrame(movies_ratings_r)

In [16]:
user_recommendation.to_csv(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
recommendation_ratings.to_csv(os.path.join(OUTPUT_PATH, OUTPUT_R_FILE))
# print(f"Successfully saved user recommendations to file: {os.path.join(OUTPUT_PATH, OUTPUT_FILE)}")

user_recommendation_r.to_csv(os.path.join(OUTPUT_PATH, OUTPUT_FILE_RATED))
recommendation_ratings_r.to_csv(os.path.join(OUTPUT_PATH, OUTPUT_R_FILE_RATED))
print(f"Successfully saved user recommendations to file: {os.path.join(OUTPUT_PATH, OUTPUT_FILE)}")

Successfully saved user recommendations to file: Output/output1.csv
